<h3 style="color: lightgreen;">Import SQL</h3>
sqlite3 is a python module that helps us interact with databases using SQL langauge

<a href="http://google.com">click here to see my notes on this...</a>

In [1]:
import sqlite3

<h3 style="color: lightgreen;">Create a Cursor Object</h3>

In [4]:
CONN = sqlite3.connect('dogs.db')
CURSOR = CONN.cursor()

<h3 style="color: lightgreen;">Dog Class</h3>

In [6]:
class Dog:
    def __init__(self, name, breed, id=None): 
        self.name = name
        self.breed = breed
        self.id = id
        
    
        

asdsf

In [15]:
def create_table(): 
        CURSOR.execute('''
            CREATE TABLE IF NOT EXISTS dogs (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                breed TEXT NOT NULL
            )
        ''')
        CONN.commit()

asdf

In [7]:
def drop_table(): 
        # Class Dog in dog.py contains method "drop_table()" that drops table "dogs" if it exists
        CURSOR.execute('DROP TABLE IF EXISTS dogs')
        CONN.commit()

asdf

In [9]:
def save(self): 
    CURSOR.execute('INSERT INTO dogs (name, breed) VALUES (?, ?)', (self.name, self.breed))
    CONN.commit()
    self.id = CURSOR.lastrowid

asdf

In [10]:
@classmethod
def create(cls, name, breed):
    dog = cls(name, breed)
    dog.save()
    return dog

asdf

In [11]:
@classmethod
def new_from_db(cls, row):
    id, name, breed = row
    dog = cls(name, breed)
    dog.id = id
    return dog

asdf

In [12]:
@classmethod
def new_from_db(cls, row):
    id, name, breed = row
    dog = cls(name, breed)
    dog.id = id
    return dog

asdf

In [13]:
@staticmethod
def get_all():
    CURSOR.execute("SELECT * FROM dogs")
    rows = CURSOR.fetchall()
    dogs = []
    for row in rows:
        dog = Dog.new_from_db(row)
        dogs.append(dog)
    return dogs

asdf

In [ ]:
@staticmethod
def get_all():
    CURSOR.execute("SELECT * FROM dogs")
    rows = CURSOR.fetchall()
    dogs = []
    for row in rows:
        dog = Dog.new_from_db(row)
        dogs.append(dog)
    return dogs

asdf

In [ ]:
@classmethod
def find_by_name(cls, name):
    CURSOR.execute('SELECT * FROM dogs WHERE name = ?', (name,))
    row = CURSOR.fetchone()
    if row:
        return cls.new_from_db(row)
    else:
        return None

asdf

In [ ]:
@classmethod
def find_by_id(cls, id):
    CURSOR.execute('SELECT * FROM dogs WHERE id=?', (id,))
    result = CURSOR.fetchone()
    if result:
        return cls.new_from_db(result)

asdf

In [ ]:
@classmethod
def find_or_create_by(cls, name, breed):
    # Check if the dog already exists in the database
    CURSOR.execute('SELECT * FROM dogs WHERE name=? AND breed=?', (name, breed))
    row = CURSOR.fetchone()
    if row:
        # If the dog already exists, return a Dog instance corresponding to that record
        return cls.new_from_db(row)
    else:
        # If the dog doesn't exist, create a new record and return a Dog instance for that record
        dog = cls(name, breed)
        dog.save()
        return dog

asdf

In [14]:
def update(self):
    if self.id is None:
        return

    CURSOR.execute('UPDATE dogs SET name = ?, breed = ? WHERE id = ?', (self.name, self.breed, self.id))
    CONN.commit()